# Cuotas pendientes → RECUPERACION DE MORA

Trabajamos con la hoja **BD CuotasPendientes**.

- **Estado:** PEN=PENDIENTE, PGD=PAGADO, APL=APLICADO, CON=CONDONADO, DEV=DEVOLUCION, MI=MORA IRRECUPERABLE.
- **Socios útiles:** solo los que tienen **25 cuotas PEN o menos**. Si tienen 26 o más PEN, no nos sirven.
- **Montos:** solo **estado = PEN** (pendiente). PGD no se usa. Solo se incluyen montos **16.95, 33.98, 38.50, 30.76**; cualquier otra cantidad se excluye.
- **Salida:** tabla para llenar el archivo **RECUPERACION DE MORA** (Fecha de cuota, Codigo asociado, nombre, VALOR, rangos de días de mora).

In [136]:
import pandas as pd
from pathlib import Path

# Ruta del Excel (está en la carpeta padre del notebook)
ruta_excel = Path().resolve().parent / 'BasesDeDatos-CUOTAS.xlsx'
print(f"Archivo: {ruta_excel}")
print(f"Existe: {ruta_excel.exists()}")

Archivo: C:\Users\adoni\Downloads\ETL-CTAS\BasesDeDatos-CUOTAS.xlsx
Existe: True


In [137]:
# Cargar datos (hoja BD CuotasPendientes)
df = pd.read_excel(ruta_excel, sheet_name='BD CuotasPendientes')
print(f"Registros totales: {len(df)}")
print(f"Columnas: {list(df.columns)}")
df.head(10)

Registros totales: 1025673
Columnas: ['socio_id', 'fecha_liquidacion', 'estado', 'anio', 'mes', 'monto', 'fecha_creacion', 'descripcion', 'Columna1', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']


,socio_id,fecha_liquidacion,estado,anio,mes,monto,fecha_creacion,descripcion,Columna1,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,6235.0,2025-01-15 11:52:08.873000,PGD,2025,2,38.50,2025-02-01 00:00:00.000,Cobro de cuota para el tipo de cuenta: Nueva M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2643.0,2025-01-15 12:06:09.399000,PGD,2025,1,30.76,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2832.0,2025-01-15 12:07:32.916000,PGD,2025,1,38.50,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4376.0,2025-01-15 12:36:10.492000,PGD,2024,12,38.50,2024-12-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3636.0,2025-01-15 15:13:16.011000,PGD,2024,12,38.50,2024-12-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1269.0,2025-01-16 10:04:08.422000,PGD,2025,1,176.00,2025-01-01 10:02:41.389,Cobro de cuota para el seguro: SOCIO 1 DEPEND ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3636.0,2025-01-15 15:13:16.011000,PGD,2025,1,38.50,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4270.0,2025-01-15 15:21:41.803000,PGD,2024,12,38.50,2024-12-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4270.0,2025-01-15 15:21:41.803000,PGD,2025,1,38.50,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5183.0,2025-01-15 15:28:03.553000,PGD,2025,1,30.76,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
# Valores en columna estado (PEN=PENDIENTE, PGD=PAGADO, APL=APLICADO, CON=CONDONADO, DEV=DEVOLUCION, MI=MORA IRRECUPERABLE)
print("Valores en estado:")
print(df['estado'].value_counts(dropna=False))

Valores en estado:
estado
PGD    560952
PEN    445587
MI      17788
CON      1334
DEL         4
pgd         3
APL         3
DEV         2
Name: count, dtype: int64


## Paso 1: Solo socios con 25 cuotas PEN o menos

Excluimos socios que tienen **26 o más** cuotas PEN (no nos sirven). Nos quedamos solo con quienes tienen **25 o menos** cuotas pendientes.

In [139]:
# Cuántas cuotas PEN tiene cada socio
cuotas_pen_por_socio = df[df['estado'] == 'PEN'].groupby('socio_id').size()
socios_irrecuperables = cuotas_pen_por_socio[cuotas_pen_por_socio >= 26].index.tolist()

print(f"Socios con 26+ cuotas PEN (excluimos): {len(socios_irrecuperables)}")
print(f"Socios con 25 o menos PEN (mantenemos): {df['socio_id'].nunique() - len(socios_irrecuperables)}")

Socios con 26+ cuotas PEN (excluimos): 2549
Socios con 25 o menos PEN (mantenemos): 4146


In [140]:
# Quitar socios con 26+ cuotas PEN
df = df[~df['socio_id'].isin(socios_irrecuperables)].copy()
df = df.reset_index(drop=True)

print(f"Registros después del Paso 1: {len(df)}")
print(f"Socios que quedan: {df['socio_id'].nunique()}")
df['estado'].value_counts(dropna=False)

Registros después del Paso 1: 528279
Socios que quedan: 4146


estado
PGD    473939
PEN     36850
MI      16446
CON      1035
DEL         4
pgd         3
APL         2
Name: count, dtype: int64

In [141]:
# Vista del resultado (primeras filas)
df.head(10)

,socio_id,fecha_liquidacion,estado,anio,mes,monto,fecha_creacion,descripcion,Columna1,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,6235.0,2025-01-15 11:52:08.873000,PGD,2025,2,38.50,2025-02-01 00:00:00.000,Cobro de cuota para el tipo de cuenta: Nueva M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2643.0,2025-01-15 12:06:09.399000,PGD,2025,1,30.76,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2832.0,2025-01-15 12:07:32.916000,PGD,2025,1,38.50,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4376.0,2025-01-15 12:36:10.492000,PGD,2024,12,38.50,2024-12-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3636.0,2025-01-15 15:13:16.011000,PGD,2024,12,38.50,2024-12-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1269.0,2025-01-16 10:04:08.422000,PGD,2025,1,176.00,2025-01-01 10:02:41.389,Cobro de cuota para el seguro: SOCIO 1 DEPEND ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3636.0,2025-01-15 15:13:16.011000,PGD,2025,1,38.50,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4270.0,2025-01-15 15:21:41.803000,PGD,2024,12,38.50,2024-12-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4270.0,2025-01-15 15:21:41.803000,PGD,2025,1,38.50,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5183.0,2025-01-15 15:28:03.553000,PGD,2025,1,30.76,2025-01-01 00:00:01.000,Cobro de cuotas para el tipo de cuenta: Nueva ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Paso 2: Montos solo por estado PEN (pendiente) y montos permitidos

Solo usamos **estado = PEN** para montos. PGD no se incluye. Además, **solo se incluyen montos** 16.95, 33.98, 38.50 y 30.76; cualquier otra cantidad se excluye de la sumatoria. Calculamos monto total por socio y monto total general (solo PEN con estos montos).

In [142]:
# Solo filas con estado PEN (pendiente) y monto permitido: 16.95, 33.98, 38.50, 30.76
MONTOS_PERMITIDOS = [16.95, 33.98, 38.50, 30.76]
df_pen_pgd = df[(df['estado'] == 'PEN') & (df['monto'].round(2).isin(MONTOS_PERMITIDOS))].copy()

# Monto total por socio (solo PEN)
monto_por_socio = df_pen_pgd.groupby('socio_id')['monto'].sum().reset_index()
monto_por_socio.columns = ['socio_id', 'monto_total']
# Última fecha_liquidacion por socio
df_pen_pgd['fecha_liquidacion'] = pd.to_datetime(df_pen_pgd['fecha_liquidacion'], errors='coerce')
ultima_fecha = df_pen_pgd.groupby('socio_id')['fecha_liquidacion'].max().reset_index()
ultima_fecha.columns = ['socio_id', 'ultima_fecha_liquidacion']
monto_por_socio = monto_por_socio.merge(ultima_fecha, on='socio_id', how='left')
monto_por_socio = monto_por_socio.sort_values('monto_total', ascending=False)

# Monto total general (solo PEN)
monto_total_general = df_pen_pgd['monto'].sum()

print(f"Registros con estado PEN: {len(df_pen_pgd)}")
print(f"Monto total general (solo PEN): {monto_total_general:,.2f}")
print(f"\nMonto total por socio (primeros 10):")
monto_por_socio.head(10)

Registros con estado PEN: 22718
Monto total general (solo PEN): 561,743.83

Monto total por socio (primeros 10):


,socio_id,monto_total,ultima_fecha_liquidacion
1644,6297.0,962.5,NaT
1658,6312.0,962.5,NaT
1691,6354.0,962.5,NaT
792,4611.0,962.5,NaT
908,4905.0,962.5,NaT
909,4907.0,962.5,NaT
787,4604.0,962.5,NaT
1249,5663.0,962.5,NaT
1273,5710.0,962.5,NaT
1423,6006.0,962.5,NaT


In [143]:
# Exportar monto por socio (solo PEN) + monto total general como última fila
ruta_monto_socio = Path().resolve().parent / 'monto_por_socio.csv'
fila_total = pd.DataFrame([{'socio_id': 'TOTAL', 'monto_total': monto_total_general, 'ultima_fecha_liquidacion': pd.NaT}])
monto_por_socio_export = pd.concat([monto_por_socio, fila_total], ignore_index=True)
try:
    monto_por_socio_export.to_csv(ruta_monto_socio, index=False, encoding='utf-8-sig')
    print(f"Montos por socio guardados en: {ruta_monto_socio}")
except PermissionError:
    ruta_alt = Path().resolve().parent / 'monto_por_socio_salida.csv'
    monto_por_socio_export.to_csv(ruta_alt, index=False, encoding='utf-8-sig')
    print(f"Archivo original abierto. Guardado en: {ruta_alt}")
print(f"Total socios: {len(monto_por_socio)} | Monto total general: {monto_total_general:,.2f}")

Montos por socio guardados en: C:\Users\adoni\Downloads\ETL-CTAS\monto_por_socio.csv
Total socios: 2242 | Monto total general: 561,743.83


## Reporte Excel: Monto total por socio y monto total general

Se genera un archivo **Reporte_Montos.xlsx** con la tabla de montos por socio y el monto total general.

In [144]:
# Generar reporte Excel: monto total por socio + monto total general
ruta_reporte = Path().resolve().parent / 'Reporte_Montos.xlsx'
try:
    with pd.ExcelWriter(ruta_reporte, engine='openpyxl') as writer:
        # Título y monto total general (primeras filas)
        pd.DataFrame([['Reporte de montos por socio (cuotas PEN)']]).to_excel(
            writer, sheet_name='Montos por socio', index=False, header=False
        )
        pd.DataFrame([['Monto total general:', round(monto_total_general, 2)]]).to_excel(
            writer, sheet_name='Montos por socio', index=False, header=False, startrow=1
        )
        # Tabla: Código socio | Monto total | Última fecha liquidación (incluye fila TOTAL al final)
        reporte_df = monto_por_socio_export.copy()
        reporte_df.columns = ['Código socio', 'Monto total', 'Última fecha liquidación']
        reporte_df.to_excel(writer, sheet_name='Montos por socio', index=False, startrow=3)
    print(f"Reporte guardado en: {ruta_reporte}")
except PermissionError:
    ruta_alt = Path().resolve().parent / 'Reporte_Montos_salida.xlsx'
    with pd.ExcelWriter(ruta_alt, engine='openpyxl') as writer:
        pd.DataFrame([['Reporte de montos por socio (cuotas PEN)']]).to_excel(
            writer, sheet_name='Montos por socio', index=False, header=False
        )
        pd.DataFrame([['Monto total general:', round(monto_total_general, 2)]]).to_excel(
            writer, sheet_name='Montos por socio', index=False, header=False, startrow=1
        )
        reporte_df = monto_por_socio_export.copy()
        reporte_df.columns = ['Código socio', 'Monto total', 'Última fecha liquidación']
        reporte_df.to_excel(writer, sheet_name='Montos por socio', index=False, startrow=3)
    print(f"Archivo original abierto. Reporte guardado en: {ruta_alt}")
print(f"  - Hoja 'Montos por socio': tabla por socio + monto total general = {monto_total_general:,.2f}")

Reporte guardado en: C:\Users\adoni\Downloads\ETL-CTAS\Reporte_Montos.xlsx
  - Hoja 'Montos por socio': tabla por socio + monto total general = 561,743.83


## Paso 3: Tabla para RECUPERACION DE MORA

Armamos la tabla con: **Fecha de cuota**, **Codigo asociado**, **nombre del asociado**, **VALOR**, y rangos de días de mora: de 0 a 30, de 31 a 60, de 61 a 90, de 91 a 120, mas de 121 días.

In [145]:
# df_pen_pgd ya tiene solo PEN. Una fila por cuota pendiente (socios con ≤25 PEN).
df_mora = df_pen_pgd.copy()

from datetime import datetime
import numpy as np
hoy = pd.Timestamp.now()

# Fecha para días de mora: usar fecha_liquidacion; si es NaT, construir desde anio/mes (día 1)
df_mora['fecha_liquidacion'] = pd.to_datetime(df_mora['fecha_liquidacion'], errors='coerce')
sin_fecha = df_mora['fecha_liquidacion'].isna()
if sin_fecha.any() and 'anio' in df_mora.columns and 'mes' in df_mora.columns:
    aux = df_mora.loc[sin_fecha, ['anio', 'mes']].rename(columns={'anio':'year', 'mes':'month'}).assign(day=1)
    df_mora.loc[sin_fecha, 'fecha_liquidacion'] = pd.to_datetime(aux)

df_mora['dias_mora'] = (hoy - df_mora['fecha_liquidacion']).dt.days
df_mora['dias_mora'] = df_mora['dias_mora'].fillna(0).clip(lower=0)

def bucket_mora(dias):
    if pd.isna(dias) or dias < 0: return 'mas_de_121'
    if dias <= 30: return 'de_0_a_30'
    if dias <= 60: return 'de_31_a_60'
    if dias <= 90: return 'de_61_a_90'
    if dias <= 120: return 'de_91_a_120'
    return 'mas_de_121'

df_mora['rango'] = df_mora['dias_mora'].apply(bucket_mora)

# Tabla RECUPERACION DE MORA con formatos: socio_id entero, nombre texto, estado texto, monto USD (2 decimales)
rec_mora = pd.DataFrame()
rec_mora['Fecha de cuota'] = df_mora['fecha_liquidacion']
rec_mora['Codigo asociado'] = df_mora['socio_id'].astype('Int64')
rec_mora['nombre del asociado'] = df_mora['socio_id'].apply(lambda x: f"Socio {int(x)}" if pd.notna(x) else "Socio ?")
rec_mora['estado'] = 'PEN'
rec_mora['VALOR'] = np.round(df_mora['monto'].astype(float), 2)
rec_mora['de 0 a 30'] = np.where(df_mora['rango'] == 'de_0_a_30', np.round(df_mora['monto'].values, 2), 0.0)
rec_mora['de 31 a 60'] = np.where(df_mora['rango'] == 'de_31_a_60', np.round(df_mora['monto'].values, 2), 0.0)
rec_mora['de 61 a 90'] = np.where(df_mora['rango'] == 'de_61_a_90', np.round(df_mora['monto'].values, 2), 0.0)
rec_mora['de 91 a 120'] = np.where(df_mora['rango'] == 'de_91_a_120', np.round(df_mora['monto'].values, 2), 0.0)
rec_mora['mas de 121 días'] = np.where(df_mora['rango'] == 'mas_de_121', np.round(df_mora['monto'].values, 2), 0.0)

print("Tabla RECUPERACION DE MORA (primeras filas). Codigo=entero, nombre=texto, estado=texto, VALOR=USD 2 decimales.")
rec_mora.head(10)

Tabla RECUPERACION DE MORA (primeras filas). Codigo=entero, nombre=texto, estado=texto, VALOR=USD 2 decimales.


,Fecha de cuota,Codigo asociado,nombre del asociado,estado,VALOR,de 0 a 30,de 31 a 60,de 61 a 90,de 91 a 120,mas de 121 días
109,2024-05-01,6913,Socio 6913,PEN,16.95,0.0,0.0,0.0,0.0,16.95
110,2024-06-01,6913,Socio 6913,PEN,16.95,0.0,0.0,0.0,0.0,16.95
111,2024-07-01,6913,Socio 6913,PEN,16.95,0.0,0.0,0.0,0.0,16.95
112,2024-08-01,6913,Socio 6913,PEN,16.95,0.0,0.0,0.0,0.0,16.95
113,2024-09-01,6913,Socio 6913,PEN,16.95,0.0,0.0,0.0,0.0,16.95
325,2024-08-01,6993,Socio 6993,PEN,16.95,0.0,0.0,0.0,0.0,16.95
326,2024-09-01,6993,Socio 6993,PEN,16.95,0.0,0.0,0.0,0.0,16.95
327,2024-10-01,6993,Socio 6993,PEN,16.95,0.0,0.0,0.0,0.0,16.95
328,2024-11-01,6993,Socio 6993,PEN,16.95,0.0,0.0,0.0,0.0,16.95
329,2024-12-01,6993,Socio 6993,PEN,16.95,0.0,0.0,0.0,0.0,16.95


In [146]:
# Exportar tabla RECUPERACION DE MORA (para copiar al archivo o usar como base)
ruta_salida = Path().resolve().parent / 'RECUPERACION_DE_MORA.csv'
try:
    rec_mora.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
    print(f"Tabla guardada en: {ruta_salida}")
except PermissionError:
    ruta_alterna = Path().resolve().parent / 'RECUPERACION_DE_MORA_salida.csv'
    rec_mora.to_csv(ruta_alterna, index=False, encoding='utf-8-sig')
    print(f"El archivo original está abierto (p. ej. en Excel). Guardado en: {ruta_alterna}")
    print("Cierre RECUPERACION_DE_MORA.csv y vuelva a ejecutar si desea sobrescribirlo.")
print(f"Total filas: {len(rec_mora)}")

Tabla guardada en: C:\Users\adoni\Downloads\ETL-CTAS\RECUPERACION_DE_MORA.csv
Total filas: 22718


## Verificación de datos

Comprobamos que el CSV y las tablas sean coherentes con los datos originales.

In [147]:
# 1. Coherencia VALOR vs columnas de rango (cada fila: VALOR = suma de los 5 rangos)
columnas_rango = ['de 0 a 30', 'de 31 a 60', 'de 61 a 90', 'de 91 a 120', 'mas de 121 días']
suma_rangos = rec_mora[columnas_rango].sum(axis=1)
ok_valor = (abs(rec_mora['VALOR'] - suma_rangos) < 0.01).all()
print(f"1. VALOR = suma de rangos por fila: {'OK' if ok_valor else 'REVISAR'}")

# 2. Número de filas: tabla RECUPERACION DE MORA = cuotas PEN (después de filtrar socios)
num_pen = df_pen_pgd[df_pen_pgd['estado'] == 'PEN'].shape[0]
print(f"2. Filas en rec_mora ({len(rec_mora)}) = cuotas PEN en df_pen_pgd ({num_pen}): {'OK' if len(rec_mora) == num_pen else 'REVISAR'}")

# 3. Monto total en rec_mora = suma de montos PEN en df_pen_pgd
suma_pen_original = df_pen_pgd[df_pen_pgd['estado'] == 'PEN']['monto'].sum()
suma_valor_csv = rec_mora['VALOR'].sum()
print(f"3. Suma VALOR en tabla ({suma_valor_csv:,.2f}) = suma montos PEN original ({suma_pen_original:,.2f}): {'OK' if abs(suma_valor_csv - suma_pen_original) < 0.01 else 'REVISAR'}")

# 4. Muestra: 3 socios al azar — monto en rec_mora vs suma por socio en datos PEN
sample_socios = rec_mora['Codigo asociado'].drop_duplicates().sample(min(3, rec_mora['Codigo asociado'].nunique()), random_state=42)
for sid in sample_socios:
    en_csv = rec_mora[rec_mora['Codigo asociado'] == sid]['VALOR'].sum()
    en_original = df_pen_pgd[(df_pen_pgd['estado'] == 'PEN') & (df_pen_pgd['socio_id'] == sid)]['monto'].sum()
    coincide = abs(en_csv - en_original) < 0.01
    print(f"   Socio {sid}: CSV={en_csv:,.2f}, original={en_original:,.2f} -> {'OK' if coincide else 'REVISAR'}")
print("4. Muestra por socio: revisar que coincidan montos arriba.")

# 5. Leer el CSV guardado y comparar totales (verificar que el archivo se escribió bien)
csv_leido = pd.read_csv(ruta_salida, encoding='utf-8-sig')
print(f"5. CSV leído: {len(csv_leido)} filas. Suma VALOR en archivo: {csv_leido['VALOR'].sum():,.2f} (debe ser {suma_valor_csv:,.2f})")

1. VALOR = suma de rangos por fila: OK
2. Filas en rec_mora (22718) = cuotas PEN en df_pen_pgd (22718): OK
3. Suma VALOR en tabla (561,743.83) = suma montos PEN original (561,743.83): OK
   Socio 5204: CSV=115.50, original=115.50 -> OK
   Socio 6477: CSV=33.90, original=33.90 -> OK
   Socio 2110: CSV=962.50, original=962.50 -> OK
4. Muestra por socio: revisar que coincidan montos arriba.
5. CSV leído: 22718 filas. Suma VALOR en archivo: 561,743.83 (debe ser 561,743.83)


In [148]:
# --- MÁS COMPROBACIONES ---

# 6. Cada fila tiene el monto en exactamente UN rango (solo una columna > 0 por fila)
rangos_positivos = (rec_mora[columnas_rango] > 0).sum(axis=1)
ok_uno_por_fila = (rangos_positivos == 1).all()
print(f"6. Una sola columna de rango con monto por fila: {'OK' if ok_uno_por_fila else 'REVISAR'} (filas con ≠1: {(rangos_positivos != 1).sum()})")

# 7. Ningún socio en rec_mora tiene más de 25 cuotas PEN
cuotas_por_socio = rec_mora.groupby('Codigo asociado').size()
max_cuotas = cuotas_por_socio.max()
socios_mas_25 = (cuotas_por_socio > 25).sum()
print(f"7. Máximo cuotas por socio = 25: {'OK' if max_cuotas <= 25 else 'REVISAR'} (máx={max_cuotas}, socios con >25: {socios_mas_25})")

# 8. Ningún socio en rec_mora está en la lista de irrecuperables (26+ PEN)
socios_en_rec = set(rec_mora['Codigo asociado'].unique())
interseccion = socios_en_rec & set(socios_irrecuperables)
print(f"8. Ningún socio irrecuperable en tabla: {'OK' if len(interseccion) == 0 else 'REVISAR'} (encontrados: {len(interseccion)})")

# 9. No hay VALOR ni montos en rangos negativos
sin_negativos = (rec_mora['VALOR'] >= 0).all() and (rec_mora[columnas_rango] >= 0).all().all()
print(f"9. Sin valores negativos en VALOR ni rangos: {'OK' if sin_negativos else 'REVISAR'}")

# 10. Suma por columna de rango en rec_mora = suma en df_mora por rango
map_col_a_rango = {'de 0 a 30': 'de_0_a_30', 'de 31 a 60': 'de_31_a_60', 'de 61 a 90': 'de_61_a_90', 'de 91 a 120': 'de_91_a_120', 'mas de 121 días': 'mas_de_121'}
print("10. Suma por rango de días (tabla vs original):")
for col in columnas_rango:
    key = map_col_a_rango[col]
    en_tabla = rec_mora[col].sum()
    en_original = df_mora[df_mora['rango'] == key]['monto'].sum()
    ok = abs(en_tabla - en_original) < 0.01
    print(f"    {col}: tabla={en_tabla:,.2f}, original={en_original:,.2f} -> {'OK' if ok else 'REVISAR'}")

# 11. Fechas de cuota: todas en el pasado (o hoy)
fechas = pd.to_datetime(rec_mora['Fecha de cuota'], errors='coerce')
futuras = (fechas > pd.Timestamp.now()).sum()
print(f"11. Fechas de cuota en el pasado: {'OK' if futuras == 0 else 'REVISAR'} (fechas futuras: {futuras})")

# 12. Resumen por rango (cantidad de cuotas y monto total por bucket)
print("12. Resumen por rango de días (cuotas y monto):")
for col in columnas_rango:
    n = (rec_mora[col] > 0).sum()
    s = rec_mora[col].sum()
    print(f"    {col}: {n} cuotas, {s:,.2f} total")

6. Una sola columna de rango con monto por fila: OK (filas con ≠1: 0)
7. Máximo cuotas por socio = 25: OK (máx=25, socios con >25: 0)
8. Ningún socio irrecuperable en tabla: OK (encontrados: 0)
9. Sin valores negativos en VALOR ni rangos: OK
10. Suma por rango de días (tabla vs original):
    de 0 a 30: tabla=123.25, original=123.25 -> OK
    de 31 a 60: tabla=106.30, original=106.30 -> OK
    de 61 a 90: tabla=321.82, original=321.82 -> OK
    de 91 a 120: tabla=251.10, original=251.10 -> OK
    mas de 121 días: tabla=560,941.36, original=560,941.36 -> OK
11. Fechas de cuota en el pasado: REVISAR (fechas futuras: 2)
12. Resumen por rango de días (cuotas y monto):
    de 0 a 30: 6 cuotas, 123.25 total
    de 31 a 60: 5 cuotas, 106.30 total
    de 61 a 90: 11 cuotas, 321.82 total
    de 91 a 120: 11 cuotas, 251.10 total
    mas de 121 días: 22685 cuotas, 560,941.36 total
